In [1]:
! pip install -q transformers
! pip install -q datasets
! pip install -q -U transformers tokenizers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.1 requires transformers<4.17,>=4.1, but you have transformers 4.17.0 which is incompatible.


In [2]:
! pip install gdown

  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=33af051c2a38b4af01ab05488b27738c104c9f447b28b047e01eff6cc4493a31
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [3]:
! pip install telegram-send

     |████████████████████████████████| 497 kB 853 kB/s            
     |████████████████████████████████| 58 kB 4.6 MB/s             
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
beatrix-jupyterlab 3.1.6 requires google-cloud-bigquery-storage, which is not installed.
gcsfs 2021.11.1 requires fsspec==2021.11.1, but you have fsspec 2022.2.0 which is incompatible.


In [4]:
%%writefile _channel.conf

[telegram]
token = 2068272694:AAFMvyFV01wXqmQUTovCZq8xCVhXDmVLggk
chat_id = -702815073

Writing _channel.conf


In [5]:
!python -m pip install --quiet telegram-send

In [6]:
!gdown --id 1-5ebQom0-51wMERqfPdUdvEgtAYXMvL1
!gdown --id 1-AJAoivJlmpPoaJDRpMboLdSeyddVT9h
!gdown --id 1-Cf7es8FrkSpXf99ZQzHwM7JhKLS8d0p

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-5ebQom0-51wMERqfPdUdvEgtAYXMvL1
To: /kaggle/working/train_feat.csv
100%|█████████████████████████████████████████| 139M/139M [00:00<00:00, 300MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-AJAoivJlmpPoaJDRpMboLdSeyddVT9h
To: /kaggle/working/val_feat.csv
100%|███████████████████████████████████████| 23.1M/23.1M [00:00<00:00, 287MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You 

In [7]:
import io
import os
import torch
import numpy as np
from functools import partial
from tqdm.notebook import tqdm
from datasets import load_metric, load_dataset, Dataset
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          AutoModel,
                          GPT2Config,
                          EvalPrediction,
                          DataCollatorWithPadding,
                          GPT2ForSequenceClassification,
                          AutoTokenizer)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

set_seed(123)
DEVICE = 'cuda' #h.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_NAME = 'sberbank-ai/rugpt3small_based_on_gpt2'
labels_ids = {'H':0, 'M':1}
N_LABELS = len(labels_ids)

In [8]:
!pip install mosestokenizer

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49188 sha256=9872b4926941cd397f2ef7ea1c2278efc8f1c35327b8200805c6bbfbe51f7bd7
  Stored in directory: /root/.cache/pip/wheels/b0/35/f7/af1258779a0b890abc3c79481460c597cb1f3659d0603cfb9d
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3354 sha256=61ef3984d4b50726f5daf3e45dc952aabaebbff0a10b97c9508bee12825e14eb
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6163 sha256=4c26b994ad3e9f9041b1b1082489e0f8de84cdd4b75d5a562c85bc5210a60918
  Stored in directory: /root/.cache/pip/wheels/fb/44/e9/914cf8fa71f0141f9314f862538d1218fcf2b94542a0fb7d35
Successfully built mosestokenizer toolwrapper uctools


In [9]:
from transformers import MarianMTModel, MarianTokenizer

# Helper function to download data for a language
def download(model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name).to('cuda')
    return tokenizer, model

tmp_lang_tokenizer, tmp_lang_model = download('Helsinki-NLP/opus-mt-ru-en')
src_lang_tokenizer, src_lang_model = download('Helsinki-NLP/opus-mt-en-ru')

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/293M [00:00<?, ?B/s]

In [10]:
def translate(texts, model, tokenizer):
    """Translate texts into a target language"""
  # Format the text as expected by the model

    batch = tokenizer(texts, return_tensors="pt", padding=True, 
                      truncation=True).to('cuda')
    generated_ids = model.generate(**batch)
    translated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return translated_texts

def back_translate(texts):
    """Implements back translation"""
  # Translate from source to target language
    translated = translate(texts, tmp_lang_model, tmp_lang_tokenizer)
  # Translate from target language back to source language
    back_translated = translate(translated, src_lang_model, src_lang_tokenizer)

    return back_translated

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('../input/ruatd2022/train.csv')

In [13]:
df_human = df.loc[df['Class'] == 'H']
human_texts = df_human.Text.tolist()

In [14]:
batches = [human_texts[x:x+20] for x in range(0, len(human_texts), 20)]

In [15]:
batches[0]

['Обустройство тротуаров, мостовых (в том числе тротуарной плиткой).',
 'Въ издержкахъ же оныхъ вы имете присылать счеты свои, какъ прямо къ намъ, такъ и къ нашему государственному казначею.',
 'Положи это туда, откуда взял.',
 'увеличение правовой грамотности и развитие правосознания граждан.',
 'Но эта разница не может рассматриваться отдельно от рекомендаций по применению респираторов.',
 'После обеда гулял со всеми, шил сапоги — плохо.',
 'при речедвигательных псевдогаллюцинациях больные ощущают, что органы речи начинают действовать помимо воли, что их языком произносят слова или целые фразы.В ряде случаев возможны комбинации различных видов псевдогаллюцинаций. Так, один из пациентов В. Х. Кандинского утверждал, что находится под наблюдением тайных агентов — «токистов», которые передают ему свои сообщения с помощью электрического тока («при этом больной должен внутренно <…> слышать то, что хотят его заставить слышать токисты» — типичный пример псевдогаллюцинаций слуха), вызывают не

In [16]:
back_translated_texts = []
for i, batch in enumerate(batches):
    back_texts = back_translate(batch)
    back_translated_texts.extend(back_texts)

In [17]:
df_human['back_translated_texts'] = back_translated_texts
df_human.to_csv('train_back_translated.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
! gzip train_back_translated.csv

In [19]:
!telegram-send --config _channel.conf --file "train_back_translated.csv.gz" --caption "Translation finished"